In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

import analysis as a

p = make_paths(a)

In [ ]:
feature_x_sample = pd.read_table(
    p['feature_x_sample_file_path'],
    index_col=0,
)

In [ ]:
for element in (
    'feature',
    'sample',
):

    if element == 'feature':

        df = feature_x_sample

        to_peek = a.FEATURES_TO_PEEK

    elif element == 'sample':

        df = feature_x_sample.T

        to_peek = a.SAMPLES_TO_PEEK

    skew_t_pdf_fit_parameter = pd.read_table(
        p['{}_skew_t_pdf_fit_parameter_file_path'.format(element)],
        index_col=0,
    )

    _1d_context_matrix = ccal.make_1d_context_matrix(
        df,
        n_job=a.MAX_N_JOB,
        skew_t_pdf_fit_parameter=skew_t_pdf_fit_parameter,
        directory_path=p['{}_directory_path'.format(element)],
    )
    
    _1d_context_matrix = pd.read_table(
        p['{}_1d_context_matrix_file_path'.format(element)],
        index_col=0,
    )

    if _1d_context_matrix.size < 1e6:

        ccal.plot_heat_map(
            _1d_context_matrix,
            title='({}) Context'.format(element),
        )

    for select_context in (
        'negative',
        'positive',
    ):

        context_summary = ccal.summarize_1d_context_matrix(
            _1d_context_matrix,
            select_context,
        )

        context_summary.sort_values(inplace=True)

        _1d_context_matrix = _1d_context_matrix.loc[context_summary.index]

        ranks = [context_summary.index.tolist().index(index) for index in context_summary.index & to_peek]

        title = '({}) {} Context Summary'.format(
            element,
            select_context,
        )

        ccal.plot_points(
            (
                'All',
                'To Peek',
            ),
            (
                tuple(range(context_summary.size)),
                ranks,
            ),
            (
                context_summary,
                context_summary[ranks],
            ),
            texts=(
                context_summary.index,
                context_summary.index[ranks],
            ),
            modes=(
                'markers',
                'markers+text',
            ),
            title=title,
            xaxis_title='Rank',
            yaxis_title=title,
        )

        n_extreme = 3

        ranks = list(range(n_extreme)) + list(range(
            -n_extreme,
            0,
        ))

        for i in ranks:

            index = context_summary.index[i]

            series = df.loc[index]

            location, scale, degree_of_freedom, shape = skew_t_pdf_fit_parameter.loc[
                index, [
                    'Location',
                    'Scale',
                    'Degree of Freedom',
                    'Shape',
                ]]

            ccal.plot_context(
                series,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                plot_rug=series.dropna().size < 1e3,
                title='({}) Context Summary Rank {} [{}]'.format(
                    element,
                    i,
                    index,
                ),
            )